In [9]:
import json

import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import numpy as np

In [2]:
all_posts = []
r = requests.get(url="https://2ch.hk/b/threads.json")
if r.status_code == 200:
    response_str = r.content.decode('utf-8')
    response = json.loads(response_str)
    thread_ids = [thread['num'] for thread in response['threads']]
    thread_ids.reverse()
    for thread_id in tqdm(thread_ids):
        r = requests.get(url=f"https://2ch.hk/b/res/{thread_id}.json")
        if r.status_code == 200:
            response_str = r.content.decode('utf-8')
            response = json.loads(response_str)
            posts = response['threads'][0]['posts']
            for post in posts:
                if 'comment' in post:
                    post_text = post['comment']
                    all_posts.append(post_text)
        else:
            raise Exception(f"Response status code is {r.status_code}")
else:
    raise Exception(f"Response status code is {r.status_code}")


162it [01:08,  2.37it/s]


In [15]:
all_posts_clean = []
for html in tqdm(all_posts):
    soup = BeautifulSoup(html, "lxml")
    for a in soup.findAll('a', href=True):
        a.extract()
    cleantext = soup.text.strip().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('  ', ' ').lower()
    if len(cleantext) > 0:
        all_posts_clean.append(cleantext)

100%|██████████| 15331/15331 [00:03<00:00, 4670.66it/s]


In [16]:
df = pd.DataFrame(all_posts_clean, columns=['text'])
df

,text
0,всю жизнь был нищим. на работу никуда не брали...
1,сказать-то что хотел?
2,и что теперь делать будешь?
3,иди нахуй наукоботское безмозглое уебище
4,жирный бототред
...,...
13339,"играю щас только на эмуляторах, щас персону 3 ..."
13340,"запускай эмуляторы компа и консолей тогда, или..."
13341,только гвинт на телефоне. захожу подари бог ра...
13342,геншин.


In [17]:
df.to_csv('data/2ch_posts.csv', index=True, index_label='id')